In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn._config import set_config
set_config(print_changed_only=False)

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer_data = load_breast_cancer()

X_features = cancer_data.data
y_label = cancer_data.target

train_X, test_X, train_y, test_y = train_test_split(X_features, y_label, test_size=0.2, random_state=42)

train_X.shape, test_X.shape

((455, 30), (114, 30))

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
ada_clf = AdaBoostClassifier(n_estimators=100)
dt_clf = DecisionTreeClassifier()

lr_final = LogisticRegression(C=10)

In [4]:
knn_clf.fit(train_X, train_y)
rf_clf.fit(train_X, train_y)
ada_clf.fit(train_X, train_y)
dt_clf.fit(train_X, train_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, random_state=None, splitter='best')

In [5]:
from sklearn.metrics import accuracy_score

knn_pred = knn_clf.predict(test_X)
rf_pred = rf_clf.predict(test_X)
ada_pred = ada_clf.predict(test_X)
dt_pred = dt_clf.predict(test_X)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(test_y, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(test_y, rf_pred)))
print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(test_y, ada_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(test_y, dt_pred)))

KNN 정확도: 0.9386
랜덤 포레스트 정확도: 0.9649
에이다부스트 정확도: 0.9737 :
결정 트리 정확도: 0.9386


In [6]:
import numpy as np

pred = np.array([knn_pred, rf_pred, ada_pred, dt_pred])
print(pred.shape)

pred = np.transpose(pred)
pred.shape

(4, 114)


(114, 4)

In [7]:
lr_final.fit(pred, test_y)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(test_y, final)))

최종 메타 모델의 예측 정확도: 0.9737


In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, train_X_n, train_y_n, test_X_n, n_folds ):
    kf = KFold(n_splits=n_folds, shuffle=False)

    train_fold_pred = np.zeros((train_X_n.shape[0] ,1 ))
    test_pred = np.zeros((test_X_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(train_X_n)):
        
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        tr_X = train_X_n[train_index]
        tr_y = train_y_n[train_index]
        te_X = train_X_n[valid_index]
        
        model.fit(tr_X, tr_y)
        train_fold_pred[valid_index, :] = model.predict(te_X).reshape(-1,1)

        test_pred[:, folder_counter] = model.predict(test_X_n)
            
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    return train_fold_pred , test_pred_mean

In [9]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, train_X, train_y, test_X, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, train_X, train_y, test_X, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, train_X, train_y, test_X, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, train_X, train_y, test_X, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [10]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, ada_train, dt_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, ada_test, dt_test), axis=1)

print('원본 학습 피처 데이터 Shape:', train_X.shape, '원본 테스트 피처 Shape:', test_X.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:', Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (455, 30) 원본 테스트 피처 Shape: (114, 30)
스태킹 학습 피처 데이터 Shape: (455, 4) 스태킹 테스트 피처 데이터 Shape: (114, 4)


In [11]:
lr_final.fit(Stack_final_X_train, train_y)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(test_y, stack_final)))

최종 메타 모델의 예측 정확도: 0.9649
